In [1]:
import Pkg; Pkg.activate(".")

include("../../TensorBenchmarks.jl")

using CairoMakie
using CliqueTrees
using DataFrames
using OMEinsumContractionOrders
using PythonCall

using OMEinsumContractionOrders: EinCode

import CSV
import KaHyPar
import Metis

using .TensorBenchmarks

const CTG = pyimport("cotengra")

  Activating project at `~/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global`
    CondaPkg Found dependencies: /Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/packages/PythonCall/mkWc2/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/environments/v1.11/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/packages/IJulia/Tb4Q4/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/richardsamuelson/.julia/artifacts/d2fecc2a9fa3eac2108d3e4d9d155e6ff5dfd0b2/bin/pixi
             │ init
             │ --format pixi
             └ /Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global/.CondaPkg
✔ Created /Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omein

Python: <module 'cotengra' from '/Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global/.CondaPkg/.pixi/envs/default/lib/python3.13/site-packages/cotengra/__init__.py'>

In [2]:
T = Tuple{Py, Py, Py}
optimizer = CTG.HyperOptimizer(; reconf_opts=pydict(), max_time="rate:1e9");

In [5]:
dir = joinpath("..", "..", "instances")
file = "gm_1fas.json"
path = joinpath(@__DIR__, dir, file)
query, matrix, weights = TensorBenchmarks.read(path)
network = make(T, query, matrix, weights);

result = solve(network, optimizer)

Python: <ContractionTree(N=177)>

In [2]:
T = Tuple{EinCode, Dict}
optimizer = HyperND(; dis = METISND(), width = 50, imbalances=130:1:130)

dir = joinpath("..", "..", "instances")
file = "mc_2022_167.json"
path = joinpath(@__DIR__, dir, file)
query, matrix, weights = TensorBenchmarks.read(path)
network = make(T, query, matrix, weights);

In [3]:
w = log2.(weights)
m = matrix * matrix'

@time treewidth(w, m; alg=SafeRules(ND(; width=50)))

  3.726396 seconds (8.15 M allocations: 617.273 MiB, 7.50% gc time, 97.59% compilation time)


6.0

In [4]:
#@time result = solve(network, optimizer)
#tc = timecomplexity(result)
#sc = spacecomplexity(result)
#tc, sc

In [7]:
import OMEinsumContractionOrders as OME

code, size_dict = network;
optimizer = Treewidth(SafeRules(ND(; width=50)))
#OME.optimize_treewidth(opt, code, size_dict)

ixs = OME.getixsv(code)
iy = OME.getiyv(code)
#OME.optimize_treewidth(opt, ixs, iy, size_dict);

L = Int

log2_edge_sizes = Dict{L,Float64}()

for (k, v) in size_dict
    log2_edge_sizes[k] = log2(v)
end

@time incidence_list = OME.IncidenceList(Dict([i=>ixs[i] for i=1:length(ixs)] ∪ [(length(ixs) + 1 => iy)]));

@time tree = OME.treewidth_method(incidence_list, log2_edge_sizes, optimizer.alg);

#@time optcode = OME.parse_eincode!(incidence_list, tree, 1:length(ixs) + 1, size_dict)[2]

#=
alg = optimizer.alg;

indices = collect(keys(incidence_list.e2v))
tensors = collect(keys(incidence_list.v2e))
weights = [log2_edge_sizes[e] for e in indices]
@time line_graph = OME.il2lg(incidence_list, indices);
=#

  0.439403 seconds (2.64 M allocations: 181.880 MiB, 4.59% gc time, 4.23% compilation time)
  8.669879 seconds (134.77 M allocations: 24.481 GiB, 25.82% gc time)
